In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
net = resnet18(pretrained=True)
net = net.to(device)
net.eval()  # Set the model to evaluation mode

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Calculate h(θ; x_i)
    output_x = net(x)
    
    # Calculate h(θ; x_i + δx_adv)
    output_x_adv = net(x + delta_x_adv)
    
    # Calculate the susceptibility ratio
    num = torch.norm(output_x - output_x_adv, p=2)
    denom = torch.norm(delta_x_adv, p=2)
    
    susceptibility = torch.exp(num / denom)
    return susceptibility.item()

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images))

# Calculate the susceptibility ratio for the batch
susceptibility = susceptibility_ratio(net, images, delta_x_adv)
print(f'Susceptibility Ratio: {susceptibility}')


Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'float' object has no attribute 'item'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from torch.autograd import *
import os
import matplotlib.pyplot as plt
import pickle
from torchvision.models import resnet18
import numpy as np
from resnet import *
from collections import OrderedDict

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)

#Load ResNet18 model with FGSM adversarial training
print('==> Loading FGSM-trained model..')
net_fgsm = ResNet18()
net_fgsm = net_fgsm.to(device)
if device == 'cuda':
    net_fgsm = torch.nn.DataParallel(net_fgsm)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_fgsmTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_fgsm.load_state_dict(new_state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_pgdTraining_iter20.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(new_state_dict)

#Load ResNet18 model with Chaos Loss minimization training
print('==> Loading ChaosLoss Minimization model..')
net_chaosLoss = ResNet18()
net_chaosLoss = net_chaosLoss.to(device)
if device == 'cuda':
    net_chaosLoss = torch.nn.DataParallel(net_chaosLoss)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_chaos_regularized_pgd_torchattacks_lambdaChaos0.8_epoch100_lr0.01.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('resnet18.'):
        name = k[9:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_chaosLoss.load_state_dict(new_state_dict)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Calculate h(θ; x_i)
    output_x = net(x)
    
    # Calculate h(θ; x_i + δx_adv)
    output_x_adv = net(x + delta_x_adv)
    
    # Calculate the susceptibility ratio
    num = torch.norm(output_x - output_x_adv, p=2)
    denom = torch.norm(delta_x_adv, p=2)
    
    susceptibility = torch.exp(num / denom)
    return susceptibility.item()

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images)) #FGSM Attack

# Calculate the susceptibility ratio for the batch
susceptibility_basic_model = susceptibility_ratio(net, images, delta_x_adv)
susceptibility_fgsm_model = susceptibility_ratio(net_fgsm, images, delta_x_adv)
susceptibility_pgd_model = susceptibility_ratio(net_pgd, images, delta_x_adv)
susceptibility_chaosLoss_model = susceptibility_ratio(net_chaosLoss, images, delta_x_adv)
print(f'Susceptibility Ratio of Basic Model: {susceptibility_basic_model}')
print(f'Susceptibility Ratio of Adversarially Trained (FGSM) Model: {susceptibility_fgsm_model}')
print(f'Susceptibility Ratio of Adversarially Trained (PGD) Model: {susceptibility_pgd_model}')
print(f'Susceptibility Ratio of Chaos-loss trained Model: {susceptibility_chaosLoss_model}')


Files already downloaded and verified
Files already downloaded and verified
==> Loading basic ResNet model..
==> Loading FGSM-trained model..
==> Loading PGD-trained model..
==> Loading ChaosLoss Minimization model..
Susceptibility Ratio of Basic Model: 1.9929317235946655
Susceptibility Ratio of Adversarially Trained (FGSM) Model: 1.0672930479049683
Susceptibility Ratio of Adversarially Trained (PGD) Model: 1.1436617374420166
Susceptibility Ratio of Chaos-loss trained Model: 1.069597601890564


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from torch.autograd import *
import os
import matplotlib.pyplot as plt
import pickle
from torchvision.models import resnet18
import numpy as np
from models import *
from collections import OrderedDict

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/models/basic_training"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/models/pgd_adversarial_training"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(new_state_dict)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Calculate h(θ; x_i)
    output_x = net(x)
    
    # Calculate h(θ; x_i + δx_adv)
    output_x_adv = net(x + delta_x_adv)
    
    # Calculate the susceptibility ratio
    num = torch.norm(output_x - output_x_adv, p=2)
    denom = torch.norm(delta_x_adv, p=2)
    
    susceptibility = torch.exp(num / denom)
    return susceptibility.item()

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images))

# Calculate the susceptibility ratio for the batch
susceptibility_basic_model = susceptibility_ratio(net, images, delta_x_adv)
susceptibility_pgd_model = susceptibility_ratio(net_chaosLoss, images, delta_x_adv)

print(f'Susceptibility Ratio of Basic Model: {susceptibility_basic_model}')
print(f'Susceptibility Ratio of Adversarially Trained (PGD) Model: {susceptibility_pgd_model}')


Files already downloaded and verified
Files already downloaded and verified
==> Loading basic ResNet model..


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer2.0.bn1.running_mean", "layer2.0.bn1.running_var", "layer2.0.conv2.weight", "layer2.0.bn2.weight", "layer2.0.bn2.bias", "layer2.0.bn2.running_mean", "layer2.0.bn2.running_var", "layer2.0.shortcut.0.weight", "layer2.0.shortcut.1.weight", "layer2.0.short

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
net = resnet18(pretrained=True)
net = net.to(device)
net.eval()  # Set the model to evaluation mode

# Define a function to register hooks and capture intermediate outputs
def register_hooks(model, layers):
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook
    
    for name, layer in model.named_modules():
        if name in layers:
            layer.register_forward_hook(get_activation(name))
    
    return activations

# Define the layers to capture
layers_to_capture = ['relu', 'layer1', 'layer2', 'layer3', 'layer4']
activations = register_hooks(net, layers_to_capture)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv, activations, layers_to_capture):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for original input
    _ = net(x)
    output_x_layers = {layer: activations[layer] for layer in layers_to_capture}
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for perturbed input
    _ = net(x + delta_x_adv)
    output_x_adv_layers = {layer: activations[layer] for layer in layers_to_capture}
    
    susceptibilities = {}
    for layer in layers_to_capture:
        output_x = output_x_layers[layer]
        output_x_adv = output_x_adv_layers[layer]
        
        # Calculate the susceptibility ratio for each layer
        num = torch.norm(output_x - output_x_adv, p=2)
        denom = torch.norm(delta_x_adv, p=2)
        susceptibility = torch.exp(num / denom)
        susceptibilities[layer] = susceptibility.item()
    
    return susceptibilities

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images))

# Calculate the susceptibility ratio for the batch
susceptibilities = susceptibility_ratio(net, images, delta_x_adv, activations, layers_to_capture)
for layer, sus in susceptibilities.items():
    print(f'Susceptibility Ratio for Layer {layer}: {sus}')


Files already downloaded and verified
Files already downloaded and verified
Susceptibility Ratio for Layer relu: 2.312288284301758
Susceptibility Ratio for Layer layer1: 5.430505275726318
Susceptibility Ratio for Layer layer2: 2.6668903827667236
Susceptibility Ratio for Layer layer3: 1.7329432964324951
Susceptibility Ratio for Layer layer4: 2.4794650077819824


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from resnet import *

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)

#Load ResNet18 model with FGSM adversarial training
print('==> Loading FGSM-trained model..')
net_fgsm = ResNet18()
net_fgsm = net_fgsm.to(device)
if device == 'cuda':
    net_fgsm = torch.nn.DataParallel(net_fgsm)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_fgsmTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_fgsm.load_state_dict(new_state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_pgdTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(new_state_dict)

# Define a function to register hooks and capture intermediate outputs
def register_hooks(model, layers):
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook
    
    for name, layer in model.named_modules():
        if name in layers:
            layer.register_forward_hook(get_activation(name))
    
    return activations

# Define the layers to capture
layers_to_capture = [
    'conv1', 'bn1',
    'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.conv2', 'layer1.0.bn2',
    'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.conv2', 'layer1.1.bn2',
    'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1',
    'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.conv2', 'layer2.1.bn2',
    'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.shortcut.0', 'layer3.0.shortcut.1',
    'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.conv2', 'layer3.1.bn2',
    'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.shortcut.0', 'layer4.0.shortcut.1',
    'layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.conv2', 'layer4.1.bn2',
    'linear'
]
activations = register_hooks(net, layers_to_capture)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv, activations, layers_to_capture):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for original input
    _ = net(x)
    output_x_layers = {layer: activations[layer] for layer in layers_to_capture}
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for perturbed input
    _ = net(x + delta_x_adv)
    output_x_adv_layers = {layer: activations[layer] for layer in layers_to_capture}
    
    susceptibilities = {}
    for layer in layers_to_capture:
        output_x = output_x_layers[layer]
        output_x_adv = output_x_adv_layers[layer]
        
        # Calculate the susceptibility ratio for each layer
        num = torch.norm(output_x - output_x_adv, p=2)
        denom = torch.norm(delta_x_adv, p=2)
        susceptibility = torch.exp(num / denom)
        susceptibilities[layer] = susceptibility.item()
    
    return susceptibilities

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images))

# Calculate the susceptibility ratio for the batch
susceptibilities_basic = susceptibility_ratio(net, images, delta_x_adv, activations, layers_to_capture)
# susceptibilities_fgsm = susceptibility_ratio(net_fgsm, images, delta_x_adv, activations, layers_to_capture)
susceptibilities_pgd = susceptibility_ratio(net_pgd, images, delta_x_adv, activations, layers_to_capture)

for layer, sus in susceptibilities_basic.items():
    print(f'Susceptibility Ratio for Layer {layer}: {sus}')
    
print(f'\n\n')
    
# for layer, sus in susceptibilities_fgsm.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')

# print(f'\n\n')

for layer, sus in susceptibilities_pgd.items():
    print(f'Susceptibility Ratio for Layer {layer}: {sus}')


Files already downloaded and verified
Files already downloaded and verified
==> Loading basic ResNet model..
==> Loading FGSM-trained model..
==> Loading PGD-trained model..
Registered hook for conv1
Registered hook for bn1
Registered hook for layer1.0.conv1
Registered hook for layer1.0.bn1
Registered hook for layer1.0.conv2
Registered hook for layer1.0.bn2
Registered hook for layer1.1.conv1
Registered hook for layer1.1.bn1
Registered hook for layer1.1.conv2
Registered hook for layer1.1.bn2
Registered hook for layer2.0.conv1
Registered hook for layer2.0.bn1
Registered hook for layer2.0.conv2
Registered hook for layer2.0.bn2
Registered hook for layer2.0.shortcut.0
Registered hook for layer2.0.shortcut.1
Registered hook for layer2.1.conv1
Registered hook for layer2.1.bn1
Registered hook for layer2.1.conv2
Registered hook for layer2.1.bn2
Registered hook for layer3.0.conv1
Registered hook for layer3.0.bn1
Registered hook for layer3.0.conv2
Registered hook for layer3.0.bn2
Registered hook 

UnboundLocalError: local variable 'layer' referenced before assignment

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from collections import OrderedDict
from resnet import *
import torchattacks

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

layers_to_capture = [
    'conv1', 'bn1',
    'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.conv2', 'layer1.0.bn2',
    'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.conv2', 'layer1.1.bn2',
    'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1',
    'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.conv2', 'layer2.1.bn2',
    'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.shortcut.0', 'layer3.0.shortcut.1',
    'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.conv2', 'layer3.1.bn2',
    'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.shortcut.0', 'layer4.0.shortcut.1',
    'layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.conv2', 'layer4.1.bn2',
    'linear'
]

# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)

#Load ResNet18 model with FGSM adversarial training
print('==> Loading FGSM-trained model..')
net_fgsm = ResNet18()
net_fgsm = net_fgsm.to(device)
if device == 'cuda':
    net_fgsm = torch.nn.DataParallel(net_fgsm)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_fgsmTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_fgsm.load_state_dict(new_state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_pgdTraining_iter20.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(new_state_dict)

#Load ResNet18 model with Chaos Loss minimization training
print('==> Loading ChaosLoss Minimization model..')
net_chaosLoss = ResNet18()
net_chaosLoss = net_chaosLoss.to(device)
if device == 'cuda':
    net_chaosLoss = torch.nn.DataParallel(net_chaosLoss)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_chaos_regularized_pgd_torchattacks_lambdaChaos0.8_epoch100_lr0.01.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('resnet18.'):
        name = k[9:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_chaosLoss.load_state_dict(new_state_dict)

# Define a function to register hooks and capture intermediate outputs
def register_hooks(model):
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook
    
    for name, layer in model.named_modules():
        layer.register_forward_hook(get_activation(name))
    
    return activations

# Register hooks for all layers
activations_basic = register_hooks(net)
activations_fgsm = register_hooks(net_fgsm)
activations_pgd = register_hooks(net_pgd)
activations_chaosLoss = register_hooks(net_chaosLoss)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, labels, activations):
    
    x = x.to(device)
    pgd = torchattacks.PGD(net, eps=0.03, alpha=0.01, steps=10)
    delta_x_adv = pgd(x, labels) - x
    delta_x_adv = delta_x_adv.to(device)
    # print(torch.norm(delta_x_adv, p=2))
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for original input
    _ = net(x)
    output_x_layers = activations.copy()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for perturbed input
    _ = net(x + delta_x_adv)
    output_x_adv_layers = activations.copy()
    
    susceptibilities = {}
    for layer in output_x_layers.keys():
        output_x = output_x_layers[layer]
        output_x_adv = output_x_adv_layers[layer]
        
        # Calculate the susceptibility ratio for each layer
        num = torch.norm(output_x - output_x_adv, p=2)
        denom = torch.norm(delta_x_adv, p=2)
        susceptibility = num / denom
        # print(num.item(), denom.item(), susceptibility.item())
        susceptibilities[layer] = susceptibility.item()
    
    return susceptibilities

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
# epsilon = 0.01
# delta_x_adv = epsilon * torch.sign(torch.randn_like(images))


# Calculate the susceptibility ratio for the batch
susceptibilities_basic = susceptibility_ratio(net, images, labels, activations_basic)
# susceptibilities_fgsm = susceptibility_ratio(net_fgsm, images, labels, activations_fgsm)
susceptibilities_pgd = susceptibility_ratio(net_pgd, images, labels, activations_pgd)
susceptibilities_chaosLoss = susceptibility_ratio(net_chaosLoss, images, labels, activations_chaosLoss)
# Print the susceptibility ratios in a table format
print("Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks:")
print("{:<25} {:<15} {:<15} {:<15} ".format('Layer', 'Basic Model', 'PGD Model', 'ChaosMin Model (λ=0.8)'))
for layer in layers_to_capture:
    sus_basic = susceptibilities_basic[layer]
    # sus_fgsm = susceptibilities_fgsm[layer]
    sus_pgd = susceptibilities_pgd[layer]
    sus_chaos = susceptibilities_chaosLoss[layer]
    print("{:<25} {:<15.6f} {:<15.6f} {:<15.6f}".format(layer, sus_basic, sus_pgd, sus_chaos))


# for layer, sus in susceptibilities_basic.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')
    
# print(f'\n\n')
    
# for layer, sus in susceptibilities_fgsm.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')

# print(f'\n\n')

# for layer, sus in susceptibilities_pgd.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')


Files already downloaded and verified
Files already downloaded and verified
==> Loading basic ResNet model..
==> Loading FGSM-trained model..
==> Loading PGD-trained model..
==> Loading ChaosLoss Minimization model..
tensor(26.6267, device='cuda:0')
tensor(26.6473, device='cuda:0')
tensor(26.6412, device='cuda:0')
Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks (new values):
Layer                     Basic Model     PGD Model       ChaosMin Model (λ=0.8) 
conv1                     2.023021        1.935358        87.573517      
bn1                       1.447865        1.477958        11.581894      
layer1.0.conv1            1.553315        1.379422        349.700378     
layer1.0.bn1              1.892387        1.701671        5.808309       
layer1.0.conv2            2.030166        1.514715        171.200638     
layer1.0.bn2              3.907794        2.823645        5.741345       
layer1.1.conv1            5.461409        2.952646       

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from collections import OrderedDict
from resnet import *
import torchattacks

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

layers_to_capture = [
    'conv1', 'bn1',
    'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.conv2', 'layer1.0.bn2',
    'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.conv2', 'layer1.1.bn2',
    'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1',
    'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.conv2', 'layer2.1.bn2',
    'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.shortcut.0', 'layer3.0.shortcut.1',
    'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.conv2', 'layer3.1.bn2',
    'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.shortcut.0', 'layer4.0.shortcut.1',
    'layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.conv2', 'layer4.1.bn2',
    'linear'
]

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

#Load ResNet18 model with PGD-7 adversarial training
print('==> Loading PGD-7-trained model..')
net_pgd7 = ResNet18()
net_pgd7 = net_pgd7.to(device)
if device == 'cuda':
    net_pgd7 = torch.nn.DataParallel(net_pgd7)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_pgdTraining.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd7.load_state_dict(new_state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-20-trained model..')
net_pgd20 = ResNet18()
net_pgd20 = net_pgd20.to(device)
if device == 'cuda':
    net_pgd20 = torch.nn.DataParallel(net_pgd20)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_pgdTraining_iter20.pth"
state_dict = torch.load(file_path)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd20.load_state_dict(new_state_dict)

# Define a function to register hooks and capture intermediate outputs
def register_hooks(model):
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook
    
    for name, layer in model.named_modules():
        layer.register_forward_hook(get_activation(name))
    
    return activations


activations_pgd7 = register_hooks(net_pgd7)
activations_pgd20 = register_hooks(net_pgd20)



# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, labels, activations):
    x = x.to(device)
    pgd = torchattacks.PGD(net, eps=0.01, alpha=0.001, steps=40)
    delta_x_adv = pgd(x, labels) - x
    delta_x_adv = delta_x_adv.to(device)

    net.eval()
    

    activations.clear()
    

    _ = net(x)
    output_x_layers = activations.copy()
    
 
    activations.clear()
    
    # Forward pass for perturbed input
    _ = net(x + delta_x_adv)
    output_x_adv_layers = activations.copy()
    
    susceptibilities = {}
    for layer in output_x_layers.keys():
        output_x = output_x_layers[layer]
        output_x_adv = output_x_adv_layers[layer]
        
        # Calculate the susceptibility ratio for each layer
        num = torch.norm(output_x - output_x_adv, p=2)
        denom = torch.norm(delta_x_adv, p=2)
        susceptibility = torch.exp(num / denom)
        susceptibilities[layer] = susceptibility.item()
    
    return susceptibilities

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
# delta_x_adv = epsilon * torch.sign(torch.randn_like(images))
# delta_x_adv = pgd_attack(net_pgd20, images, labels)

# Calculate the susceptibility ratio for the batch
susceptibilities_pgd_iter20 = susceptibility_ratio(net_pgd20, images, labels, activations_pgd20)
susceptibilities_pgd_iter7 = susceptibility_ratio(net_pgd7, images, labels, activations_pgd7)

# Print the susceptibility ratios in a table format
print("Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks:")
print("{:<25} {:<15} {:<15}".format('Layer', 'PGD-7 Model', 'PGD-20 Model'))
for layer in layers_to_capture:
    sus_pgd7 = susceptibilities_pgd_iter7[layer]
    sus_pgd20 = susceptibilities_pgd_iter20[layer]
    print("{:<25} {:<15.6f} {:<15.6f}".format(layer, sus_pgd7, sus_pgd20))



# for layer, sus in susceptibilities_basic.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')
    
# print(f'\n\n')
    
# for layer, sus in susceptibilities_fgsm.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')

# print(f'\n\n')

# for layer, sus in susceptibilities_pgd.items():
#     print(f'Susceptibility Ratio for Layer {layer}: {sus}')


Files already downloaded and verified
Files already downloaded and verified
==> Loading PGD-7-trained model..
==> Loading PGD-20-trained model..
Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks:
Layer                     PGD-7 Model     PGD-20 Model   
conv1                     4.984600        5.071968       
bn1                       3.353668        3.333042       
layer1.0.conv1            2.675270        2.601351       
layer1.0.bn1              3.208047        3.204614       
layer1.0.conv2            3.206050        2.983945       
layer1.0.bn2              8.270608        7.808430       
layer1.1.conv1            11.650421       8.006520       
layer1.1.bn1              8.050145        5.739753       
layer1.1.conv2            3.923095        3.252265       
layer1.1.bn2              29.485292       19.955465      
layer2.0.conv1            7.155480        6.200864       
layer2.0.bn1              5.906315        5.351881       
layer2.0.conv

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChaosRegularizedLoss(nn.Module):
    def __init__(self, lambda_chaos=0.1, perturbation_magnitude=1e-2):
        super(ChaosRegularizedLoss, self).__init__()
        self.lambda_chaos = lambda_chaos
        self.perturbation_magnitude = perturbation_magnitude
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, model, inputs, targets):
        # Standard task-specific loss
        outputs = model(inputs)
        task_loss = self.cross_entropy_loss(outputs, targets)

        # Chaos regularization term
        perturbation = self.perturbation_magnitude * torch.randn_like(inputs)
        perturbed_inputs = inputs + perturbation
        outputs_perturbed = model(perturbed_inputs)

        hidden_states = model.get_hidden_states(inputs)
        hidden_states_perturbed = model.get_hidden_states(perturbed_inputs)
        
        chaos_loss = torch.mean((hidden_states - hidden_states_perturbed).pow(2))

        # Combined loss
        total_loss = task_loss + self.lambda_chaos * chaos_loss
        return total_loss

# Example usage
# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

file_path = "/home/pratyush/pratyushg/resnetTraining/resnet18_cifar10_basicTraining.pth"
state_dict = torch.load(file_path)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net.load_state_dict(new_state_dict)

model = net
criterion = ChaosRegularizedLoss(lambda_chaos=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Get a batch of training data
dataiter = iter(trainloader)
inputs, targets = dataiter.next()
optimizer.zero_grad()
loss = criterion(model, inputs, targets)
loss.backward()
optimizer.step()


==> Loading basic ResNet model..


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Step 1: Set Up Environment and Import Libraries
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 2: Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Step 3: Define the ResNet-18 Model
class ResNet18WithHidden(nn.Module):
    def __init__(self):
        super(ResNet18WithHidden, self).__init__()
        self.resnet18 = resnet18(pretrained=False, num_classes=10)
        self.hidden_states = None

    def forward(self, x):
        # Extracting hidden states from one of the intermediate layers
        def hook(module, input, output):
            self.hidden_states = output

        # Register hook to a layer (e.g., the layer before the last fully connected layer)
        handle = self.resnet18.layer4[1].conv2.register_forward_hook(hook)
        
        output = self.resnet18(x)
        
        # Remove the hook after forward pass to avoid accumulating hooks
        handle.remove()
        
        return output

    def get_hidden_states(self):
        return self.hidden_states

model = ResNet18WithHidden().to(device)

# Step 4: Define the Custom Chaos Regularized Loss
class ChaosRegularizedLoss(nn.Module):
    def __init__(self, lambda_chaos=0.1, perturbation_magnitude=1e-2):
        super(ChaosRegularizedLoss, self).__init__()
        self.lambda_chaos = lambda_chaos
        self.perturbation_magnitude = perturbation_magnitude
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, model, inputs, targets):
        # Compute standard task-specific loss
        outputs = model(inputs)
        task_loss = self.cross_entropy_loss(outputs, targets)

        # Generate small random perturbation for inputs
        perturbation = self.perturbation_magnitude * torch.randn_like(inputs).to(inputs.device)
        perturbed_inputs = inputs + perturbation

        # Forward pass for perturbed inputs
        _ = model(perturbed_inputs)

        # Get hidden states for both original and perturbed inputs
        hidden_states = model.get_hidden_states().detach()
        _ = model(inputs)  # Forward pass again to get hidden states for original inputs
        hidden_states_perturbed = model.get_hidden_states().detach()
        
        # Chaos regularization term (mean squared difference normalized by perturbation size)
        chaos_loss = torch.mean((hidden_states - hidden_states_perturbed).pow(2) / perturbation.pow(2).mean())
        
        # Combined loss
        total_loss = task_loss + self.lambda_chaos * chaos_loss
        return total_loss

criterion = ChaosRegularizedLoss(lambda_chaos=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train the Model
for epoch in range(20):  # Adjust the number of epochs as needed
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        
        loss = criterion(model, inputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the trained model
torch.save(model.state_dict(), 'resnet18_cifar10_chaos_regularized.pth')

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
[Epoch 1, Batch 100] loss: 1.842
[Epoch 1, Batch 200] loss: 1.584
[Epoch 1, Batch 300] loss: 1.465
[Epoch 2, Batch 100] loss: 1.320
[Epoch 2, Batch 200] loss: 1.254
[Epoch 2, Batch 300] loss: 1.203
[Epoch 3, Batch 100] loss: 1.122
[Epoch 3, Batch 200] loss: 1.127
[Epoch 3, Batch 300] loss: 1.109
[Epoch 4, Batch 100] loss: 1.043
[Epoch 4, Batch 200] loss: 1.069
[Epoch 4, Batch 300] loss: 1.043
[Epoch 5, Batch 100] loss: 1.072
[Epoch 5, Batch 200] loss: 1.039
[Epoch 5, Batch 300] loss: 1.084
[Epoch 6, Batch 100] loss: 1.103
[Epoch 6, Batch 200] loss: 1.098
[Epoch 6, Batch 300] loss: 1.124
[Epoch 7, Batch 100] loss: 1.111
[Epoch 7, Batch 200] loss: 1.174
[Epoch 7, Batch 300] loss: 1.215
[Epoch 8, Batch 100] loss: 1.201
[Epoch 8, Batch 200] loss: 1.284
[Epoch 8, Batch 300] loss: 1.309
[Epoch 9, Batch 100] loss: 1.316
[Epoch 9, Batch 200] loss: 1.511
[Epoch 9, Batch 300] loss: 1.398
[Epoch 10, Batch 100] loss: 1.458

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

# Define the device to use for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformations for the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Load ResNet18 model
net = model
net = net.to(device)
net.eval()  # Set the model to evaluation mode

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Calculate h(θ; x_i)
    output_x = net(x)
    
    # Calculate h(θ; x_i + δx_adv)
    output_x_adv = net(x + delta_x_adv)
    
    # Calculate the susceptibility ratio
    num = torch.norm(output_x - output_x_adv, p=2)
    denom = torch.norm(delta_x_adv, p=2)
    
    susceptibility = torch.exp(num / denom)
    return susceptibility.item()

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images))

# Calculate the susceptibility ratio for the batch
susceptibility = susceptibility_ratio(net, images, delta_x_adv)
print(f'Susceptibility Ratio: {susceptibility}')


Files already downloaded and verified
Files already downloaded and verified
Susceptibility Ratio: 1.2778984308242798
